In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Define the ticker symbol

ticker = 'AAPL'  # Apple Inc.
#ticker = "MSFT"  # Microsoft


# Get the data
data = yf.download(ticker, start='2008-01-01', end='2024-08-01')

# Save to CSV
data.to_csv('stock_data.csv')


[*********************100%***********************]  1 of 1 completed


In [ ]:
# Load data
data = pd.read_csv('stock_data.csv', index_col='Date', parse_dates=True)
data.fillna(method='ffill', inplace=True)

# Visualize the Dataset
data

<ipython-input-21-f752d7244f1a>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-01-02,7.116786,7.152143,6.876786,6.958571,5.876341,1079178800
2008-01-03,6.978929,7.049643,6.881786,6.961786,5.879056,842066400
2008-01-04,6.837500,6.892857,6.388929,6.430357,5.430276,1455832000
2008-01-07,6.473214,6.557143,6.079643,6.344286,5.357590,2072193200
2008-01-08,6.433571,6.516429,6.100000,6.116071,5.164871,1523816000
...,...,...,...,...,...,...
2024-07-25,218.929993,220.850006,214.619995,217.490005,217.238556,51391200
2024-07-26,218.699997,219.490005,216.009995,217.960007,217.708008,41601300
2024-07-29,216.960007,219.300003,215.750000,218.240005,217.987686,36311800


In [ ]:
# Scale input features
scaler_X = MinMaxScaler()
scaled_features = scaler_X.fit_transform(data[['Open', 'High', 'Low', 'Close', 'Volume']])


In [ ]:
# Scale target variable
scaler_y = MinMaxScaler()
scaled_close = scaler_y.fit_transform(data[['Close']])


In [ ]:
sequence_length = 60
X = []
y = []

for i in range(sequence_length, len(scaled_features)):
    X.append(scaled_features[i-sequence_length:i])
    y.append(scaled_close[i])  # Use the scaled 'Close' price as target

X = np.array(X)
y = np.array(y)


In [ ]:
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-Head Attention
    attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(inputs, inputs)
    attention = Dropout(dropout)(attention)
    attention = LayerNormalization(epsilon=1e-6)(attention + inputs)

    # Feed Forward Network
    ff = Dense(ff_dim, activation='relu')(attention)
    ff = Dense(inputs.shape[-1])(ff)
    ff = Dropout(dropout)(ff)
    outputs = LayerNormalization(epsilon=1e-6)(ff + attention)
    return outputs

input_shape = X_train.shape[1:]  # (sequence_length, num_features)
inputs = Input(shape=input_shape)
x = transformer_encoder(inputs, head_size=64, num_heads=4, ff_dim=128, dropout=0.1)
x = GlobalAveragePooling1D()(x)
outputs = Dense(1)(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=50,
    batch_size=64,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
)


Epoch 1/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 7s 79ms/step - loss: 0.1987 - val_loss: 0.0516
Epoch 2/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 96ms/step - loss: 0.0030 - val_loss: 0.0675
Epoch 3/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 90ms/step - loss: 0.0020 - val_loss: 0.0653
Epoch 4/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - loss: 0.0018 - val_loss: 0.0533
Epoch 5/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - loss: 0.0015 - val_loss: 0.0540
Epoch 6/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - loss: 0.0015 - val_loss: 0.0476
Epoch 7/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - loss: 0.0013 - val_loss: 0.0488
Epoch 8/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - loss: 0.0011 - val_loss: 0.0445
Epoch 9/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 102ms/step - loss: 0.0011 - val_loss: 0.0492
Epoch 10/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - loss: 0.0011 - val_loss: 0.0446
Epoch 11/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step - loss: 9.1859e-04 - val_loss: 0.0399
Epoch 12/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - loss

In [ ]:
predictions = model.predict(X_test)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step


In [ ]:
# Inverse transform predictions and y_test
predictions_inverse = scaler_y.inverse_transform(predictions)
y_test_inverse = scaler_y.inverse_transform(y_test)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

rmse = np.sqrt(mean_squared_error(y_test_inverse, predictions_inverse))
mape = mean_absolute_percentage_error(y_test_inverse, predictions_inverse) * 100

print(f'RMSE: {rmse}')
print(f'MAPE: {mape}%')


RMSE: 65.40345695621623
MAPE: 40.25434669369431%
